# Status

## Problems

1. Very poor and inconsistent performance on operate-first
2. nootbook output cells data is painful in git consider other options:
   - https://mg.readthedocs.io/git-jupyter.html
   - https://gist.github.com/pbugnion/ea2797393033b54674af

## Next Steps

1. Work on improving Dockerfiles to avoid unnecessary layers
2. Exploit startup hooks to try and customize the machine a bit more:
    - add correct user name to /etc/passwd
    - see if we can figure out a way to rename home dir
    - fix env then USER and HOME, etc
    - fix hostname
3. document that we are using the default jupyter user jovyan (https://jupyter.readthedocs.io/en/latest/community/content-community.html#what-is-a-jovyan) to be our container user

    

## Working and notes  
0.  This url works locally if you have your jupyter server running locally:<br> (eg make nb on container repo)
    http://127.0.0.1:8888/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2FL00_210_JA.ipynb&branch=main
1.  Operate First direct spawn url for bu-cs-jupyter-book:latest:<br> 
https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/spawn?bu-cs-jupyter-book:latest
2. To get nbgitpuller to clone or sync the UndertheCovers repo:<br>
https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/user-redirect/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2FL00_210_JA.ipynb&branch=main
  - Other very usefull urls:
      - authenticated users can see the Jupyterhub namespace on openshift here:<br>
https://console-openshift-console.apps.zero.massopen.cloud/k8s/ns/opf-jupyterhub/pods <br>
      - Your pod can be found here: <br>
https://console-openshift-console.apps.zero.massopen.cloud/k8s/ns/opf-jupyterhub/pods/jupyterhub-nb-jappavoo-40bu-2eedu <br>
This can give you a bit more info on the state of your pod
3. 2 required futzing with the image so that the group id of the joyvan home directory is root (see [dockerfile](https://github.com/jappavoo/bu-cs-book-dev/blob/c88bc65d65b334150724981dad145f395e177505/Dockerfile#L87)): <br>
``` shell
# as a hack we are going to try changing group id of /home/joyvan to be root to see if I can trick things into
# working on the moc
RUN chgrp -R root /home/jovyan
```
The startup logic of the container which we inherited from the jupyter/minimal-notebook:latest (https://github.com/jupyter/docker-stacks/blob/master/base-notebook/start.sh) does a bunch magic:
  - when spawned by the main jupyterhub server on oprate-first the container seems to launch with a userid of the operate-first uesr (eg in my case 1000950000) and a gid of root (0)
  - which does not match the uid of the joyvan user in the container (1000) so the startup script replaces the joyvan user with a new joyvan user who's uid is the operate-first user id.  
  - however it does nothing to the permissions of the homedir and it seems to let the gid of root persist
  - so by default the home directory is readable but can not be written
  - to fix this I modify the container's base permission of the joyvan homedir to be group 0
  - oddly enough this all seems to work
4. Oddly enough this also seems to solve my problem of persistent state:  The default logic is starting the jupyter server in the joyvan home dir which is not on the volume mounted by operate-first and its contents is server instance specific.  Destroying the server (stopping it) and restarting seeds the joyvan home dir back to its initial state from the container
5. To access the api we must have the api token this however is not present in the same way as it was when running the jupyter notebook server locally -- see ln_preamble.py:<br>
   on operate-first jupyerhub the api token is in an environment variable
6. To make get requests we must use a full url to the localhost introduced localhost_url in ln_preamble
7. To access the users server urls we need to prefix with `/hub/user-redirect` or any equivalent eg. to get access terminal 3 it would be `/hub/user-redirect`.  So now to embed a terminal we would can succesfully do the following: <br>
``` python
IFrame('/hub/user-redirect/terminals/2', 1400, 600)
```
or
``` python
IFrame('/user/jappavoo@bu.edu/terminals/2', 1400, 600)
```
or
``` python
IFrame('/hub/user/jappavoo@bu.edu/terminals/2', 1400, 600)
```
which means that we can use base_url as reported by server info in both environments to embedded a terminal
``` python
IFrame(base_url + 'terminals/2', 1400, 600)
```
The above now lets us embed terminals in the notebooks similar to the functionality of running a locally hosted version of the container and starting the jupyter notebook server by hand (see make devnb).  Have rewriten python/ln_preamble.py appropriately.
8. Had a very hard time with getting python package installed and configure in a way that worked on opf with all constraints meet.  The current head of the bu-cs-book-dev repo has the version that I found to work:  
    - boots on opf (from some reason prior commit in which I very carefully tested python package commands fails to boot).  
    - all jupyter notebook extensions that I want installed by default work and are on when user opens a notebook
    - the right version of jupyter books is installed -- use of conda got the wrong version switch to using pip install and this got the right version
9. cleanup home dir and added stuff to bash rc to have better default behaviour


# Rough notes and playing around

Operate first url to directly spawn bu image

https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/spawn?bu-cs-jupyter-book:latest

Working with Tom on operate first slack the following url git clone 


https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/user-redirect/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2FL00_210_JA.ipynb&branch=main


How to set 
```
c.Spawner.default_url = '/data/jupyterhub/jupyterhub_notebooks'
c.Spawner.notebook_dir = '/data/jupyterhub/jupyterhub_notebooks'
```

Hmmm it now seems that we are launching bu-cs container and home dir ends up being joyvan need to look at start up script as there is a lot of logic in it to adjust user and group

At this point the running joyvan user on operatefirst seems to be some system assigned id with group root which leads to not having the right permissions to write home dir

For the moment a hack might be to make the joyvan homdir and subdirs group root


Have managed to get the notebook running and now stuggling to get embedded terminals work.

1) calling the api to create the terminals works after getting the api token from the environment variables
2) however running into security setting need to be fixed
  ```
  
  c.Spawner.args = ['--NotebookApp.tornado_settings={"headers":{"Content-Security-Policy": "frame-ancestors * self host_ip:port"}}']   

c.JupyterHub.tornado_settings = { 'headers': { 'Content-Security-Policy': "frame-ancestors * self host_ip:port"} }
```
see: 
- https://github.com/jupyterhub/jupyterhub/issues/379#issuecomment-353607675
- https://github.com/jupyterhub/the-littlest-jupyterhub/issues/312#issuecomment-590679516


In [1]:
%run -i underthecovers/python/ln_preamble.py

In [2]:
print(base_url)

/


In [3]:
showET()

https://jupyterhub.github.io/nbgitpuller/link?hub=https://127.0.0.1